In [20]:
from INEvalidador import Validador
from INEvalidador import conexionSQL
import pandas as pd
import os
import re

In [14]:
# Función para convertirlas todas las columnas de la base a mayuscula
def columnas_a_mayuscula(tabla:str):
    base = pd.read_feather(tabla)
    columnas_originales = base.columns
    columnas_nuevas = []
    for columna in columnas_originales:
        col = columna.upper()
        columnas_nuevas.append(col)
    diccionario = dict(zip(columnas_originales, columnas_nuevas))
    base = base.rename(columns=diccionario)
    return base

In [30]:
# Función para hacer hacer las bases de datos con las que se trabajarán
def unir_tablas(ronda: int):
    if ronda == 1:
        ruta = "Bases/Ronda1" 
        archivos = os.listdir(ruta)
        bases = []
        for arch in archivos:
            if arch != 'audio_pr.feather' and arch != 'personas.feather':
                bases.append(pd.read_feather(ruta + arch))
        db_hogares1 = bases[0]
        for base in bases[1:]:
            db_hogares1.merge(db_hogares1, base, on='level-1-id', how='outer')
        db_hogares1.to__feather("Bases/Ronda1/HogaresRonda1.feather")
        db_personas1 = pd.read("Bases/Ronda1/personas.feather")
        db_personas1.name = "PersonasRonda1.feather"
    else:
        ruta = "Bases/Ronda2" 
        archivos = os.listdir(ruta)
        bases = []
        for arch in archivos:
            if arch != 'audios.feather' and arch != 'personas_sr.feather':
                bases.append(pd.read_feather(ruta + arch))
        db_hogares2 = bases[0]
        for base in bases[1:]:
            db_hogares2.merge(db_hogares2, base, on='level-1', how='outer')
        db_hogares2.to_feather("Bases/Ronda2/HogaresRonda2.feather")
        db_personas2 = pd.read_feather("Bases/Ronda2/personas_sr.feather")
        db_personas2.name = "PersonasRonda2.feather"

In [13]:
# Función que obtenga la primera variable en la validación para poder identificar qué pasa usará
def identificar_base_a_usar(condicion: str):
    variable = re.findall(r'^\s*([A-Za-z_][A-Za-z\d_]*)\s*[<>!=]*', condicion, re.MULTILINE)
    return variable